# **Before each experiment**

Select `Kernel` > `Restart Kernel and Clear All Outputs`

In [ ]:
%%cmd
poetry update

***
# **Without mouse on stage**

In [2]:
import np_workflows
import np_services
import np_workflows.npxc as npxc
import np_workflows.experiments.task_trained_network as TTN

np_workflows.elapsed_time_widget()

Not connected to a rig: `OpenEphys.host` needs to be set manually.


Error importing np_services.proxies: Rig index not specified and not running on a rig.


ImportError: cannot import name 'Sync' from 'np_services' (c:\Users\ben.hardcastle\AppData\Local\pypoetry\Cache\virtualenvs\np-workflows-bIqIfclO-py3.11\Lib\site-packages\np_services\__init__.py)

***
## Quiet mode
**on**  [*default*]
- error details are hidden
- regular messages displayed (log level = INFO)

**off**
- full error details (with traceback)
- extra messages displayed (log level = DEBUG)

In [ ]:
np_workflows.quiet_mode_widget()

***
## Launch apps via RSC
[optional]

In [ ]:
np_services.start_rsc_apps()

***
## Select mouse and user

In [ ]:
user, mouse = np_workflows.user_and_mouse_widget()

***
## Check MTrain and select session
Re-run cell this cell if mouse ID is changed

In [ ]:
np_workflows.mtrain_widget(mouse)
selected_session = TTN.stim_session_select_widget(mouse)

***
## Generate new session
Check mouse ID and session are correct: this cell will lock them in!

In [ ]:
experiment: np_workflows.WithLims = TTN.new_experiment(mouse, user, selected_session.session)

platform_json = experiment.platform_json
hab = isinstance(experiment, TTN.Hab)

***
## Checks before starting

# Ephys day 2?

## **Don't forget to adjust probe targeting!**

In [ ]:
np_workflows.check_hardware_widget()

In [ ]:
if not hab:
    np_workflows.check_openephys_widget()

***
## Setup, test, reset all components
[critical]

In [ ]:
experiment.initialize_and_test_services()

***
## Dip probes

In [ ]:
if not hab:
    np_workflows.di_widget(platform_json)
    np_workflows.photodoc_widget('pre_experiment_surface_image')

***
***
# **With mouse on stage**
## Before lowering cartridge

In [ ]:
platform_json.HeadFrameEntryTime = npxc.now()
np_workflows.wheel_height_widget(platform_json)
np_workflows.check_mouse_widget()

***
## When cartridge is lowered

In [ ]:
platform_json.CartridgeLowerTime = npxc.now()
np_workflows.photodoc_widget('brain_surface_image')

***
## Probe insertion

In [ ]:
if not hab:
    platform_json.ProbeInsertionStartTime = npxc.now()
    np_workflows.isi_widget(mouse.lims, colormap=False)

***
## Photodoc before advancing probes

In [ ]:
if not hab:
    np_workflows.photodoc_widget('pre_insertion_surface_image')

***
## Settle timer

In [ ]:
if not hab:
    np_workflows.print_countdown_timer(minutes=.1 if experiment.ttn_session.value == 'pretest' else 10)

***
## Photodoc after probes settled, before experiment

In [ ]:
if not hab:
    np_workflows.photodoc_widget('post_insertion_surface_image')

In [1]:
np_workflows.pre_stim_check_widget()

NameError: name 'np_workflows' is not defined

***
## Start devices recording

In [ ]:
experiment.start_recording()

***
## Start stimulus scripts
[mapping | main | opto]

In [ ]:
experiment.run_stim_scripts()

***
## Stop recording

In [ ]:
experiment.stop_recording_after_stim_finished()

***
## Before removing probes

In [ ]:
if not hab:
    np_workflows.photodoc_widget('post_stimulus_surface_image')

***
## After fully retracting probes

In [ ]:
np_workflows.photodoc_widget('post_experiment_surface_image')

***
## After raising cartridge

In [1]:
platform_json.HeadFrameExitTime = npxc.now()

np_workflows.finishing_checks_widget()

NameError: name 'npxc' is not defined

## Finalize, copy data

In [ ]:
platform_json.workflow_complete_time = npxc.now()
platform_json.write()

experiment.finalize_services(*experiment.recorders, *experiment.stims)
experiment.validate_services(*experiment.recorders, *experiment.stims)
experiment.copy_files()